# Hand Gesture Recognition
## Part of Final Project to control Turtlbot Lite 3 via Hand Gesture
## Authors: Archit Jain, Vishal Nadig

In [1]:
#Importing libraries
import os
from sklearn.model_selection import train_test_split
import pandas as pd 
import cv2

In [2]:
data_dir = fr"C:\Users\archi\Downloads\grayscale_gesture"

In [3]:
# Loop through each subdirectory in the data directory
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

data = []
labels = []
filenames_ls = []

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

dim = (250, 250)

for label in os.listdir(data_dir):
    # Get the full path of the subdirectory
    label_dir = os.path.join(data_dir, label)
    label = int(label)
    # Loop through each image file in the subdirectory
    for filename in os.listdir(label_dir):
        # Get the full path of the image file
        img_path = os.path.join(label_dir, filename)
        data_aux = []
        x_ = []
        y_ = []
        img = cv2.imread(img_path)
        image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # convert to RGB
        # image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
        results = hands.process(image)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for id, lm in enumerate(hand_landmarks.landmark):
                    h, w, c = image.shape
                    x = hand_landmarks.landmark[id].x
                    y = hand_landmarks.landmark[id].y

                    x_.append(x)
                    y_.append(y)
                
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_))
                    data_aux.append(y - min(y_))


                filenames_ls.append(filename)
                labels.append(label)      
                data.append(data_aux)

In [4]:
df = pd.DataFrame({
    "filename" : filenames_ls,
    'label' : labels,
    'data' : data
    })



df.to_csv("ASL_Grey_landmarks_new.csv")


In [5]:
df.head()

,filename,label,data
0,zero_111.jpg,0,"[0.0, 0.08762304484844208, 0.04669642448425293..."
1,zero_1110.jpg,0,"[0.0, 0.26034096628427505, 0.09558302164077759..."
2,zero_1111.jpg,0,"[0.0, 0.5345455408096313, 0.10513269901275635,..."
3,zero_1112.jpg,0,"[0.03647130727767944, 0.624834269285202, 0.127..."
4,zero_1113.jpg,0,"[0.035660624504089355, 0.6398380994796753, 0.1..."


## Splitting Data into Train and test with 80% in train

In [6]:
# df['landmarks_cx'] = df['landmarks_cx'].apply(lambda x: np.array(x))
# df['landmarks_cy'] = df['landmarks_cy'].apply(lambda x: np.array(x))

train_data = df.sample(frac=0.8, random_state=42)
val_data = df.drop(train_data.index)

print(train_data.shape)
print(val_data.shape)

(9044, 3)
(2261, 3)


In [7]:
cx_cols = ["cx" + str(i) for i in range(1, 22)]
cy_cols = ["cy" + str(i) for i in range(1, 22)]
label_cols = ["label" + str(i) for i in range(0, 10)]
print(label_cols)

['label0', 'label1', 'label2', 'label3', 'label4', 'label5', 'label6', 'label7', 'label8', 'label9']


In [8]:
# Define the input and output for the model
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

## Model 1 - Training Convulational Neural Network from Scratch

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
 
model = RandomForestClassifier()

model.fit(x_train, y_train)

y_predict = model.predict(x_test)

score = accuracy_score(y_predict, y_test)

print('{}% of samples were classified correctly !'.format(score * 100))

f = open('Sign_Language_Grey_classifier_2.p', 'wb')
pickle.dump({'model': model}, f)
f.close()

96.99248120300751% of samples were classified correctly !
